importing the necessary libraries

In [ ]:
import numpy
import pandas
import joblib

from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
import numpy as np

reading the data and encoding the labels for better performence

In [ ]:
with open('mfccdata2.npy', 'rb') as f:
    data_set=np.load(f)
print("Dataset shape:", data_set.shape)

Dataset shape: (1000, 20, 130)


In [ ]:
with open('labels2.npy', 'rb') as f:
    labels=np.load(f)
print("labels shape:", labels.shape)

labels shape: (1000,)


In [ ]:
encoder = LabelEncoder()
labels = encoder.fit_transform(labels)

In [ ]:
print(labels.shape)
print(data_set.shape)

(1000,)
(1000, 20, 130)


In [ ]:
x=data_set
y=labels

In [ ]:
print("recommended number of hidden layers",int(800/(2*(10+36))))

recommended number of hidden layers 8


Random Forset Classifier

In [ ]:
rf = RandomForestClassifier(n_estimators=150)
scores = cross_val_score(rf, x, y, cv=5)
print("Accuracy: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))

Support Vector Classifer

In [ ]:
svc = SVC(C=35, gamma=0.08)
scores = cross_val_score(svc, x, y, cv=5)
print("Accuracy: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))

Accuracy: 0.57 (+/- 0.07)


KNN Classifer 

In [ ]:
knn = KNeighborsClassifier(n_neighbors=6)
scores = cross_val_score(knn, x, y, cv=5)
print("Accuracy: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))

Accuracy: 0.46 (+/- 0.07)


ANN with Adam optimizer and drop out

In [ ]:
from keras import models
from keras import layers
from keras import Input
from keras.constraints import maxnorm
from keras.optimizers import adam
from keras.applications import VGG16
from keras.utils.np_utils import to_categorical
from tensorflow.keras import backend as K

Using TensorFlow backend.


In [ ]:
scores = []
for i in range(20) : 
  X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.2)
  model = models.Sequential()
  model.add(layers.Dense(256, activation='relu',input_shape=(X_train.shape[1],)))
  model.add(layers.Dense(128, activation='relu'))
  model.add(layers.Dropout(0.2))
  model.add(layers.Dense(64, activation='relu',kernel_constraint=maxnorm(3)))
  model.add(layers.Dense(10, activation='softmax'))

  ada = adam(learning_rate=0.007, beta_1=0.9, beta_2=0.999, epsilon=1e-07)

  model.compile(optimizer= ada, loss='sparse_categorical_crossentropy', metrics=['accuracy'])
  model.fit(X_train, y_train, epochs=20, batch_size=70,verbose=0)

  test_loss, test_acc = model.evaluate(X_test,y_test)
  scores+=[test_acc]
scores=np.array(scores)
print("Accuracy: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))
#best accuracy so far is Dense NN

Classification using CNN

Import mfcc data :

In [ ]:
with open('mfccdata.npy', 'rb') as f:
    data_set=np.load(f)
print("Dataset shape:", data_set.shape)

Dataset shape: (1000, 128, 130)


In [ ]:
with open('labels2.npy', 'rb') as f:
    labels=np.load(f)
print("labels shape:", labels.shape)

labels shape: (1000,)


label encoding :

In [ ]:
x = data_set

encoder = LabelEncoder()
labels = encoder.fit_transform(labels)

y = labels
y = to_categorical(y,num_classes=10)

defining parameters :

In [ ]:
SEED = 42
CONV_FILTER_COUNT = 256
BATCH_SIZE = 32
EPOCH_COUNT = 100
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.2)
n_features = X_train.shape[2]
input_shape = (None, n_features)
model_input = Input(input_shape, name='input')
layer = model_input

Importing libraries :

In [ ]:
from keras.models import Sequential
from keras.layers import Activation
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import Flatten
from keras.layers import Conv1D
from keras.layers import MaxPooling1D
from keras.layers import Conv2D
from keras.layers import MaxPooling2D
from keras.layers import AveragePooling2D
from keras.layers import Input
from keras.layers import GRU
from keras.layers import LSTM
from keras.layers import concatenate
from keras.models import Model
from keras.applications.vgg16 import VGG16

The architecture :

In [ ]:
scores = []
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.2)
model = Sequential()
input = shape=x.shape[1:]
model.add(Conv2D(24, (5, 5), strides=(1, 1), input_shape=input))
model.add(AveragePooling2D((2, 2), strides=(2,2)))
model.add(Activation('relu'))

model.add(Conv2D(48, (5, 5), padding="same"))
model.add(AveragePooling2D((2, 2), strides=(2,2)))
model.add(Activation('relu'))

model.add(Conv2D(48, (5, 5), padding="same"))
model.add(AveragePooling2D((2, 2), strides=(2,2)))
model.add(Activation('relu'))

model.add(Flatten())
model.add(Dropout(rate=0.5))

model.add(Dense(64))
model.add(Activation('relu'))
model.add(Dropout(rate=0.5))

model.add(Dense(10))
model.add(Activation('softmax'))


ada = adam(learning_rate=0.01, beta_1=0.9, beta_2=0.999, epsilon=1e-07)

model.compile(optimizer= ada, loss='categorical_crossentropy', metrics=['accuracy'])
model.fit(X_train, y_train, epochs=EPOCH_COUNT, batch_size=BATCH_SIZE,verbose=1)

test_loss, test_acc = model.evaluate(X_test,y_test)
print("test loss :",test_loss)
print("test_accu :",test_acc)

In [ ]:
model.fit(data_x, y)
joblib.dump(model, 'model.pkl')
print("Trained and saved the model to project folder successfully.")